In [1]:
# Importazione delle librerie necessarie
import os
import sys
import random
import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sys import platform
# Impostazione dei percorsi
_base_path = '\\'.join(os.getcwd().split('\\')[:-1]) + '\\' if platform =='win32' else '/'.join(os.getcwd().split('/')[:-1]) + '/'
sys.path.append(_base_path)
# Importare le librerie necessarie
from monai.utils import set_determinism
from src.helpers.config import get_config
from src.models.gnn import GraphSAGE, GAT, ChebNet
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.utils import k_hop_subgraph, to_networkx
from sklearn.linear_model import Ridge
import networkx as nx

In [2]:
# Definizione dei percorsi
_config = get_config()
data_path = os.path.join(_base_path, _config.get('DATA_FOLDER'))
graph_path = os.path.join(data_path, _config.get('GRAPH_FOLDER'))
saved_path = os.path.join(_base_path, _config.get('SAVED_FOLDER'))
reports_path = os.path.join(_base_path, _config.get('REPORT_FOLDER'))
logs_path = os.path.join(_base_path, _config.get('LOG_FOLDER'))
cache_path = os.path.join(_base_path, 'cache')
os.makedirs(cache_path, exist_ok=True)
if platform == 'win32':
    data_path = data_path.replace('/', '\\')
    graph_path = graph_path.replace('/', '\\')
    saved_path = saved_path.replace('/', '\\')
    reports_path = reports_path.replace('/', '\\')
    logs_path = logs_path.replace('/', '\\')

In [3]:
# Impostare un seed per la riproducibilità
set_determinism(seed=3)
random.seed(3)
np.random.seed(3)
torch.manual_seed(3)

In [4]:
# Definizione dei parametri del modello
# PARAMETRI CONDIVISI
num_node_features = 50 # Dimensione feature di input
num_classes = 4 # Numero di classi di output (0=Non-tumore,1=NCR, 2=ED, 3=ET)
lr = 1e-4 # Learning rate per l'ottimizzatore
weight_decay = 1e-5 # Weight decay per l'ottimizzatore
dropout = .0 # Probabilità di dropout (per features)
hidden_channels = [512, 512, 512, 512, 512, 512, 512] # Unità nascoste
# PARAMETRI SPECIFICI PER IL MODELLO CHEBNET
k = 4 # Ordine polinomiale Chebyshev

In [5]:
# Creazione del modello da utilizzare (ChebNet)
model = ChebNet(
in_channels=num_node_features,
hidden_channels=hidden_channels,
out_channels=num_classes,
dropout=dropout,
K=k
)
print(f"Modello creato: {model.__class__.__name__}")

Modello creato: ChebNet


In [6]:
# Funzione per trovare e caricare un grafo per l'analisi
def find_and_load_graph(subject_id=None):
    """
    Trova e carica un grafo per l'analisi.
    Args:
    subject_id: ID specifico del soggetto da caricare, se None ne verrà scelto uno casualmente
    Returns:
    data: Il grafo caricato
    subject_id: L'ID del soggetto caricato
    """
    if subject_id is None:
        # Trova le cartelle dei soggetti che contengono grafi
        subject_dirs = [d for d in os.listdir(graph_path) if os.path.isdir(os.path.join(graph_path, d))]
        valid_subjects = []
# Cerca i primi 10 soggetti che hanno file .graph
        for subject in subject_dirs[:100]: # Limita la ricerca per efficienza
            graph_file = os.path.join(graph_path, subject, f"{subject}.graph")
            if os.path.isfile(graph_file):
                valid_subjects.append(subject)
                if len(valid_subjects) >= 10:
                    break
        if not valid_subjects:
            raise FileNotFoundError("Nessun grafo trovato nella directory data/graphs/")
# Scegli un soggetto casuale
        subject_id = random.choice(valid_subjects)
# Carica il grafo
    graph_file = os.path.join(graph_path, subject_id, f"{subject_id}.graph")
    if not os.path.isfile(graph_file):
        raise FileNotFoundError(f"File grafo non trovato per il soggetto{subject_id}")
    print(f"Caricamento grafo: {graph_file}")
    data = torch.load(graph_file)
    return data, subject_id

In [7]:
# Carica un grafo specifico con alta accuratezza
subject_id = "BraTS-GLI-01166-000" # Grafo con accuratezza 100% (lo stesso del notebook 4 per confronto)
try:
    data, subject_id = find_and_load_graph(subject_id)
    print(f"Grafo caricato con successo: {subject_id}")
    print(f"Numero di nodi: {data.x.shape[0]}")
    print(f"Numero di archi: {data.edge_index.shape[1]}")
    print(f"Numero di features per nodo: {data.x.shape[1]}")
except FileNotFoundError as e:
    print(f"Errore: {e}")
    print("Tentativo di caricamento di un grafo alternativo...")
    data, subject_id = find_and_load_graph(None)
    print(f"Grafo alternativo caricato: {subject_id}")

Caricamento grafo: /home/gianuca/Scrivania/Tesi/Progetto/brain-tumor-graph-segmentation-main/brain-tumor-graph-segmentation-main/data/graphs/BraTS-GLI-01166-000/BraTS-GLI-01166-000.graph
Grafo caricato con successo: BraTS-GLI-01166-000
Numero di nodi: 2607
Numero di archi: 26070
Numero di features per nodo: 50


/tmp/ipykernel_298044/1901407787.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(graph_file)


In [8]:
# Carica il modello pre-addestrato
model_files = [f for f in os.listdir(saved_path) if 'CHEBNET' in f and f.endswith('_best.pth')]
if not model_files:
    raise FileNotFoundError("Nessun modello ChebNet pre-addestrato trovato nella directory saved/")
latest_model = model_files[-1]
print(f"Utilizzo del modello pre-addestrato: {latest_model}")
model.load_state_dict(torch.load(os.path.join(saved_path, latest_model),map_location=torch.device('cpu')))
model.eval()

Utilizzo del modello pre-addestrato: CHEBNET_1739029370_best.pth


/tmp/ipykernel_298044/4237471290.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(saved_path, latest_model),map_location=tor

ChebNet(
  (layers): ModuleList(
    (0): ChebConv(50, 512, K=4, normalization=sym)
    (1-6): 6 x ChebConv(512, 512, K=4, normalization=sym)
    (7): ChebConv(512, 4, K=4, normalization=sym)
  )
  (dropout): Dropout(p=0.0, inplace=False)
)

In [9]:
# Verifica dell'accuratezza sul grafo caricato
with torch.no_grad():
    outputs = model(data.x, data.edge_index.type(torch.int64))
    predicted_labels = outputs.argmax(dim=1)
    # Crea un tensore delle classi da usare, contenente le predizioni del modello
    node_classes = predicted_labels.clone()
    # Calcola l'accuratezza usando predicted_labels e data.y se possibile
    try:
        accuracy = (predicted_labels == data.y).float().mean().item()
        print(f"Accuratezza sul grafo {subject_id}: {accuracy:.4f}")
    except Exception as e:
        print(f"Impossibile calcolare l'accuratezza usando data.y: {e}")
        print("Usando solo le predizioni per l'analisi")
    class_counts = torch.bincount(predicted_labels, minlength=num_classes)
    print(f"Distribuzione classi predette: {class_counts.numpy()}")

Accuratezza sul grafo BraTS-GLI-01166-000: 1.0000
Distribuzione classi predette: [2550   16    4   37]


In [10]:
def gnn_explainer_simple(node_idx, x, edge_index):
    """
    Implementazione semplificata di GNNExplainer basata sui gradienti.
    """
    # Assicurati che node_idx sia un tensore 1D e mantieni una versione scalare
    if isinstance(node_idx, int):
        node_idx_tensor = torch.tensor([node_idx], dtype=torch.int64)
        node_idx_scalar = node_idx
    elif isinstance(node_idx, torch.Tensor) and node_idx.dim() == 0:
        node_idx_tensor = node_idx.unsqueeze(0)
        node_idx_scalar = node_idx.item()
    else:
        node_idx_tensor = node_idx.to(torch.int64)
        node_idx_scalar = node_idx.item() if node_idx.numel() == 1 else node_idx[0].item()
    # Clona i dati di input e abilita il calcolo dei gradienti
    x_grad = x.clone().detach().requires_grad_(True)
    # Forward pass - AGGIUNGI .type(torch.int64) a edge_index
    with torch.enable_grad():
        outputs = model(x_grad, edge_index.type(torch.int64))
        pred_class = outputs[node_idx_scalar].argmax().item()
        # Calcola il gradiente rispetto alla classe predetta
        model.zero_grad()
        outputs[node_idx_scalar, pred_class].backward()
        # Usa il gradiente delle feature del nodo come misura di importanza
        node_importance = x_grad.grad[node_idx_scalar].abs()
        return node_importance, pred_class
def gradcam_explainer(node_idx, x, edge_index):
    """
    Implementazione di GradCAM per GNN.
    Args:
    node_idx: Indice del nodo da spiegare
    x: Feature dei nodi
    edge_index: Indici degli archi
    Returns:
    node_importance: Importanza delle feature
    pred_class: Classe predetta
    """
    # Assicurati che node_idx sia un tensore 1D e mantieni una versione scalare
    if isinstance(node_idx, int):
        node_idx_tensor = torch.tensor([node_idx], dtype=torch.int64)
        node_idx_scalar = node_idx
    elif isinstance(node_idx, torch.Tensor) and node_idx.dim() == 0:
        node_idx_tensor = node_idx.unsqueeze(0)
        node_idx_scalar = node_idx.item()
    else:
        node_idx_tensor = node_idx.to(torch.int64)
        node_idx_scalar = node_idx.item() if node_idx.numel() == 1 else node_idx[0].item()
    # Simile a GNNExplainer ma con pesi diversi
    x_grad = x.clone().detach().requires_grad_(True)
    with torch.enable_grad():
        outputs = model(x_grad, edge_index.type(torch.int64))
        pred_class = outputs[node_idx_scalar].argmax().item()
        # Backpropagation
        model.zero_grad()
        outputs[node_idx_scalar, pred_class].backward()
        # GradCAM pondera i gradienti
        gradients = x_grad.grad[node_idx_scalar]
        node_importance = gradients * x[node_idx_scalar] # Moltiplica per l'attivazione
        node_importance = node_importance.abs()
    return node_importance, pred_class